# Census Income
---

#### Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import set_config
set_config(transform_output="pandas")

#### Import the data file

In [2]:
raw_df = pd.read_csv('files/datasets/adult.csv')
raw_df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


Because values from the CSV file in Kaggle uses `?` as null values, we will replace those question marks with a NaN. 

In [3]:
raw_df.replace(to_replace='?', value=np.nan, inplace=True)

In [4]:
raw_df.replace(to_replace='<=50K', value=0, inplace=True)
raw_df.replace(to_replace='>50K', value=1, inplace=True)

#### View information about the dataframe

In [5]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       30725 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education.num   32561 non-null  int64 
 5   marital.status  32561 non-null  object
 6   occupation      30718 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital.gain    32561 non-null  int64 
 11  capital.loss    32561 non-null  int64 
 12  hours.per.week  32561 non-null  int64 
 13  native.country  31978 non-null  object
 14  income          32561 non-null  int64 
dtypes: int64(7), object(8)
memory usage: 3.7+ MB


In [6]:
raw_df.isnull().sum()

age                  0
workclass         1836
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     583
income               0
dtype: int64

#### Split the data into target and features

In [7]:
def xy_split(dataframe, y_column):
    features = dataframe.loc[:, dataframe.columns != y_column]
    target = dataframe[y_column]
    return features, target

X, y = xy_split(raw_df, 'income')

##### Create training and testing sets

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##### Create a copy of the original dataset

In [9]:
df = X_train.copy()

---

# Data Preparation

#### Determine which columns have low cardinality
We will start fixing these columns first

In [10]:
def ohe_eligibility(datafame):
    for i in datafame.columns:
        if len(datafame[i].value_counts()) < 10:
            print(f'{len(datafame[i].value_counts())} - {i}')
        else:
            pass
        
ohe_eligibility(df)

8 - workclass
7 - marital.status
6 - relationship
5 - race
2 - sex


----

# Data Cleaning

In [11]:
import pandas as pd
from sklearn.preprocessing import FunctionTransformer

# Define a custom function to replace a value in a dataframe
def replace_values(dataframe, column, replacements_dict):
    dataframe[column] = dataframe[column].replace(replacements_dict)
    return dataframe

### Fixing Marital Status column

Reduce categories by merging different categories into one.

Single: **Never Married**

Break: **Divorced**, **Widowed**, **Separated**

Married: **Married-civ-spouse**, **Married-spouse-absent**, **Married AF spouse**

In [12]:
replacements_dict = {"Never-married": "single", "Divorced": "break", "Separated": "break", "Widowed": "break", 
                     "Married-civ-spouse": "married", "Married-spouse-absent": "married", "Married-AF-spouse": "married"}

replace_transformer = FunctionTransformer(replace_values, kw_args={'column': 'marital.status', 'replacements_dict': replacements_dict})

# df_new = replace_transformer.transform(df)

----

### Fixing Workclass column

Reduce categories by merging different categories into one.

P.Employed: **Private, Without-pay**

S.Employed: **Self-emp-not-inc, Self-emp-inc**

G.Employed: **Local-gov, State-gov, Federal-gov**

Unemployed

In [13]:
replacements_dict_wc = {"Private": "P.Employed", "Without-pay": "P.Employed", "Self-emp-not-inc": "S.Employed", 
                        "Self-emp-inc": "S.Employed", "Local-gov": "G.Employed", "State-gov": "G.Employed", 
                        "Federal-gov": "G.Employed", "Never-worked": "Unemployed"}

replace_transformer_wc = FunctionTransformer(replace_values, kw_args={'column': 'workclass', 'replacements_dict': replacements_dict_wc})

# df_new = replace_transformer_wc.transform(df)

### Fixing Relationship column (NA)

Reduce categories by merging different categories into one.

parent: **Husband, Wife**

no_family: **Not-in-family, Own-child, Unmarried, Other-relative**

In [14]:
replacements_dict_r = {"Husband": "parent", "Wife": "parent", "Not-in-family": "no_family", 
                       "Own-child": "no_family", "Unmarried": "no_family", "Other-relative": "no_family"}

replace_transformer_r = FunctionTransformer(replace_values, kw_args={'column': 'relationship', 'replacements_dict': replacements_dict_r})

# df_new = replace_transformer_r.transform(df)

---

### Fixing Occupation column

#### Domain Knowledge: http://www.socsci.uci.edu/~mhuffman/occ_codes.pdf

Reduce categories by merging different categories into one.

managerial/professional: **Prof-specialty, Exec-managerial**

technical/sales/administrative: **Adm-clerical, Sales, Tech-support**

service: **Protective-serv, Priv-house-serv**

operator/fabricator/laborer: **Machine-op-inspct, Transport-moving, Handlers-cleaners**

farming/forestry/fishing: **Farming-fishing** 

precision/production/craft/repair: **Craft-repair**

military: **Armed-Forces**

In [15]:
replacements_dict_o = {"Prof-specialty": "managerial/professional", "Exec-managerial": "managerial/professional", "Adm-clerical": "technical/sales/administrative", "Sales": 
                       "technical/sales/administrative", "Tech-support": "technical/sales/administrative", "Protective-serv": "service", "Priv-house-serv": "service",
                       "Machine-op-inspct": "operator/fabricator/laborer", "Transport-moving": "operator/fabricator/laborer", "Handlers-cleaners": "operator/fabricator/laborer"}

replace_transformer_o = FunctionTransformer(replace_values, kw_args={'column': 'remainder__occupation', 'replacements_dict': replacements_dict_o})

# df_new = replace_transformer_o.transform(df)

---

### Apply function transformers as for loop

In [16]:
transformers = [(replace_transformer_r, 'relationship', replacements_dict_r),    
                (replace_transformer_wc, 'workclass', replacements_dict_wc),    
                (replace_transformer, 'marital.status', replacements_dict)]

for transformer, column, replacements_dict in transformers:
    replace_transformer = FunctionTransformer(replace_values, kw_args={'column': column, 'replacements_dict': replacements_dict})

In [17]:
# Data Before

raw_df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,NaN,77053,HS-grad,9,Widowed,NaN,Not-in-family,White,Female,0,4356,40,United-States,0
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,0
2,66,NaN,186061,Some-college,10,Widowed,NaN,Unmarried,Black,Female,0,4356,40,United-States,0
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,0
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,0


In [18]:
df_transformed = replace_transformer.fit_transform(raw_df)

In [19]:
# Data after, it works

df_transformed.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,NaN,77053,HS-grad,9,break,NaN,Not-in-family,White,Female,0,4356,40,United-States,0
1,82,Private,132870,HS-grad,9,break,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,0
2,66,NaN,186061,Some-college,10,break,NaN,Unmarried,Black,Female,0,4356,40,United-States,0
3,54,Private,140359,7th-8th,4,break,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,0
4,41,Private,264663,Some-college,10,break,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,0


* [ ] Age
* [ ] workclass
* [ ] fnlwght
* [ ] education
* [ ] education.num
* [X] martial.status
* [ ] occupation
* [X] relationship
* [ ] race
* [X] sex
* [ ] occupation
* [ ] capital.gain
* [ ] capital.loss
* [ ] hours.per.week
* [ ] native.country
* [ ] income

### Try

Idiota, kung titignan mo yung underlying pattern kung bakit wala yung occupation, you can see na halos wala din yung workclass kapag NA yung occupation. So it is valid na tanggalin nalang natin yung rows na wala both sa workclass and occupation.


Lesson Learned: Before imputing missing values, determine the type of missing data